In [ ]:
import json
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 文件夹路径
folder_path = "imgs"

# 读取文件夹中的所有图像文件和JSON文件
image_files = glob.glob(os.path.join(folder_path, "*.jpg"))
json_files = glob.glob(os.path.join(folder_path, "*.json"))

# 遍历图像文件和JSON文件
for image_file, json_file in zip(image_files, json_files):
    # 读取图像
    image = cv2.imread(image_file)

    # 读取JSON数据
    with open(json_file, 'r') as f:
        json_data = json.load(f)

    # 遍历JSON数据中的标注信息
    for shapes in json_data['shapes']:
        label =shapes['label']
        points = np.array(shapes['points'], np.int32)

        # 绘制蓝色多边形框选标注为'1'的区域
        if label == '1':
            cv2.polylines(image, [points], isClosed=True, color=(255, 0, 0), thickness=2)

        # 绘制红色框选标注为'0'的区域
        if label == '0':
            x, y, w, h = cv2.boundingRect(points)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cropped_image = image[y:y+h, x:x+w]
            resized_image = cv2.resize(cropped_image, (400, 400))

            # 保存缩放后的框选区域
            output_image_file = os.path.splitext(image_file)[0] + '_cropped_resized.jpg'
            output_image_file = output_image_file.replace("imgs", "face4")
            cv2.imwrite(output_image_file, resized_image)
    # 显示图像
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
import cv2
import json
import os
import numpy as np
import matplotlib.pyplot as plt

# 文件夹路径
folder_path = "imgs"

# 加载文件夹下的所有图片文件名和JSON标注文件名
image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]
json_files = [f[:-4] + ".json" for f in image_files]

for image_file, json_file in zip(image_files, json_files):
    # 加载图像
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)

    # 加载JSON标注数据
    json_path = os.path.join(folder_path, json_file)
    if not os.path.isfile(json_path):
        continue
    with open(json_path, 'r') as f:
        json_data = json.load(f)

    # 解析JSON数据
    annotations = json_data['shapes']
    for annotation in annotations:
        label =annotation['label']
        points = np.array(annotation['points'], dtype=np.int32)

        if label == '1':
            cv2.polylines(image, [points], isClosed=True, color=(255, 0, 0), thickness=2)
            # 绘制多边形边界框
            cv2.polylines(image, [points], isClosed=True, color=(255, 0, 0), thickness=2)
    
            # 提取多边形边界框内的图像区域
            mask = np.zeros_like(image)
            cv2.fillPoly(mask, [points], color=(255, 255, 255))
            masked_image = cv2.bitwise_and(image, mask)
            masked_image = cv2.resize(masked_image, (800, 600))
            # 显示提取的全身图片
            plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))
            plt.title("Full body")
            plt.show()
    
        # # 显示带有多边形边界框的原始图像
        # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        # plt.title("Original image")
        # plt.show()